In [ ]:
import numpy as np

In [ ]:
class Gaussian_total:
    def __init__(self,Gaussian_mean =[],Gaussian_var =[],Gaussian_weight =[],number_of_Gaussian = 0):
        self.Gaussian_mean =[]
        self.Gaussian_var =[]
        self.Gaussian_weight =[]
        self.number_of_Gaussian = number_of_Gaussian
class HMM_total:
    def __init__(initial_matrix=[], transition_cost =[], state_mix =[],state_number = 0 ):
        self.initial_matrix=initial_matrix
        self.transition_cost=transition_cost
        self.state_mix=state_mix

In [ ]:
def gaussian(data,sigma_square,ftheta):
    sigma_square_producted=np.prod(sigma_square,axis=1)
    p=(1/(np.sqrt((2*np.pi)**2 *sigma_square)))*(np.exp(-0.5*np.sum((data-ftheta)**2/sigma_square_producted,axis=1)))
    return p
def log_gaussian(data,sigma_square,ftheta):
    cost= 0.5*np.sum(np.log((2*np.pi)*(sigma_square)),axis=1)+0.5*np.sum(np.square((ftheta-data))/sigma_square,axis=1)
    return cost
def mix_log_gaussian(mix_gaussian,ftheta):
    Gaussian_mean=mix_gaussian.Gaussian_mean
    Gaussian_var=mix_gaussian.Gaussian_var
    Gaussian_weight=mix_gaussian.Gaussian_weight
    cost=log_gaussian(Gaussian_mean,Gaussian_var,ftheta)
    changed_cost=np.sum(Gaussian_weight*cost)
    return changed_cost
def mix_gaussian(mix_gaussian,ftheta):
    cost=0
    for i in range(0,mix_gaussian.number_of_Gaussian):
        Gaussian_mean=mix_gaussian.Gaussian_mean[i][:]
        sigma_square=mix_gaussian.Gaussian_var[i][:]
        Gaussian_weight=mix_gaussian.Gaussian_weight[i]
        cost0=gaussian(Gaussian_mean,sigma_square,ftheta)
        cost=cost+Gaussian_weight*cost0
    return cost

In [ ]:
def GMM_HMM_DTW(HMM_total,data):
    mark=False
    transition_cost=HMM_total.transition_cost
    temp_list0=np.zeros([39])
    temp_list1=zp.zeros([39])
    mixture_of_all_states=[]
    new_GMM=Gaussian_total()
    new_GMM.Gaussian_mean.append(zeros)
    new_GMM.Gaussian_var.append(ones)
    new_GMM.Gaussian_weight=[1]
    new_GMM.Num_of_Gaussian = 1
    mix_of_all_states.append(new_GMM)
    for mix in HMM_total.state_mix:
        mix_of_all_states.append(mix)
    data=np.vstack([zeros,data])
    len_mix_of_all_states=len(mix_of_all_states)
    len_data=len(data)
    P=np.zeros([len_mix_of_all_states,len_data])
    # Finding minimum-cost path
    for j in range(0,len_data):
      for i in range(len_mix_of_all_states):
            C_ij= mixture_log_gaussian(mix_of_all_states[i],data[j])
            if i-2>=0:
                P[i][j]=min(P[i][j-1]+transition_cost[i][i],P[i-1][j-1]+transition_cost[i-1][i],
                            P[i-2][j-1]+transition_cost[i-2][i])+C_ij
            elif i-1>=0:
                P[i][j]=min(P[i][j-1]+transition_cost[i][i],P[i-1][j-1]+transition_cost[i-1][i])+C_ij
            else:
                P[i][j]=P[i][j]+C_ij
    P=P/len_data
    distance=P[-1][-1]
    if mark:
        return distance,traceback(P)
    else:
        return distance